In [7]:
import graphene
import requests
import json
import pandas as pd

In [4]:
import graphene

class Query(graphene.ObjectType):
    hello = graphene.String(name=graphene.String(default_value="World"))

    def resolve_hello(self, info, name):
      return 'Hello ' + name

schema = graphene.Schema(query=Query)
result = schema.execute('{ hello }')
print(result.data['hello']) # "Hello World"

Hello World


# Functions

In [86]:
def vote_json_to_df(vote_json):
    for vote in vote_json['data']['votes']:
        id = vote['id']
        ipfs = vote['ipfs']
        voter = vote['voter']
        created = vote['created']
        space = vote['space']['id']
        proposal = vote['proposal']['id']
        choice = vote['choice']
        #metadata = vote['metadata']: {},
        vp = vote['vp']
        vp_by_strategy = tuple(vote['vp_by_strategy'])
        vp_state = vote['vp_state']
        row_as_series  = pd.Series(
            data=(
                id,
                ipfs,
                voter,
                created,
                space,
                proposal,
                choice,
                #metadata,
                vp,
                vp_by_strategy,
                vp_state,
                ),
            index=(
                'id',
                'ipfs',
                'voter',
                'created',
                'space',
                'proposal',
                'choice',
                #'metadata',
                'vp',
                'vp_by_strategy',
                'vp_state',
            )
        )
        row_as_series_list.append(row_as_series)
    return_df = pd.DataFrame(row_as_series_list)
    return return_df
    

In [92]:
proposal_query = '''
query Proposals {
  proposals (
    first: 10000000
    where: {
      space_in: ["index-coop.eth"],
      state: "closed"
    },
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    title
    body
    choices
    start
    end
    snapshot
    state
    author
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    votes
    space {
      id
      name
    }
  }
}
'''


In [93]:
print(proposal_query)


query Proposals {
  proposals (
    first: 10000000
    where: {
      space_in: ["index-coop.eth"],
      state: "closed"
    },
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    title
    body
    choices
    start
    end
    snapshot
    state
    author
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    votes
    space {
      id
      name
    }
  }
}



In [94]:
url = 'https://hub.snapshot.org/graphql/'
r = requests.post(url, json={'query': proposal_query})


In [95]:
print(r.status_code,r.reason)

200 OK


In [96]:
request_json = r.json()

In [97]:
len(request_json['data']['proposals'])

221

In [98]:
with open('all_proposals_20211119.json', 'w') as f:
    json.dump(request_json, f)

In [99]:
request_json['data']['proposals'][0]

{'id': '0x940fea8c7965da18b12fc6a081478204d41cda975ca8acdf220a17b0b99193aa',
 'title': 'IIP-105 DG1: Launch MATIC 2x Flexible Leverage Index (Polygon)',
 'body': "This is the first of two votes required for a new product to be onboarded to the Index Coop.\n\nA passing vote here signals the Index Coop's intent to further research the MATIC 2x Flexible Leverage Index (Polygon) feasibility resulting in a product prioritization score.\n\nPassing Thresholds:\n\n* 5% or more of circulating supply must participate (2,181,385*0.05 = 109 069)\n* 50% or more of participating tokens must vote FOR\n\nFOR - Pass MATIC 2x Flexible Leverage Index (Polygon) Index through Decision Gate 1. Begin work to research & assign MATIC 2x Flexible Leverage Index (Polygon) a product prioritization score.\n\nAGAINST - Do not pass MATIC 2x Flexible Leverage Index (Polygon) Index through Decision Gate 1.\n\nDG1 is a signaling step to gauge community sentiment. The full-text MATIC 2x Flexible Leverage Index (Polygon)

In [88]:
for proposal in request_json['data']['proposal']:
    print(proposal['state'],proposal['choices'])

KeyError: 'proposal'

In [74]:
vote_query = '''
query Votes {
  votes (
    first: 9999999
    skip: 0
    where: {
      proposal: "QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa"
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    ipfs
    voter
    created
    space {
      id
    }
    proposal {
      id
    }
    choice
    metadata
    vp
    vp_by_strategy
    vp_state
  }
}
'''


# Jank

In [82]:
url = 'https://hub.snapshot.org/graphql/'
r = requests.post(url, json={'query': vote_query})
print(r.status_code,r.reason)

200 OK


In [83]:
vote_json = r.json()

In [84]:
vote_json['data']['votes']

[{'id': '0x142418f28d21b3ba356caa91025d45ee84558dde1e7af7b9042e00f27bf83048',
  'ipfs': 'QmYpBuRKXYdLfgaCMKnzuwqEBmj9EijWTpNL5hnmzr6oxG',
  'voter': '0x53C40473DcdFd927C4201cCFE24E314a7D7C3584',
  'created': 1635623597,
  'space': {'id': 'index-coop.eth'},
  'proposal': {'id': 'QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa'},
  'choice': 1,
  'metadata': {},
  'vp': 88.85845661274872,
  'vp_by_strategy': [88.85845661274872, 0],
  'vp_state': 'final'},
 {'id': '0x7b696f7df1ec720a1b976d06ffed895e8b7ed0cb8872392f54ad221b3e10eb92',
  'ipfs': 'QmUrs4tt8a49ZQPasxXvrQ2xGGWb9ZhSUTBU7rqMEE8KZj',
  'voter': '0x0100e4f2a841e773564c333f889ab6D6Bd5FcB22',
  'created': 1635602262,
  'space': {'id': 'index-coop.eth'},
  'proposal': {'id': 'QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa'},
  'choice': 1,
  'metadata': {},
  'vp': 8912.113703479938,
  'vp_by_strategy': [753.0289283021797, 8159.084775177758],
  'vp_state': 'final'},
 {'id': '0x1ead0f3d9ebd31a303db747fd12b89b7c701ec4d2d56b5eddcd82ae39ab

In [78]:
def vote_json_to_df(vote_json):
    for vote in vote_json['data']['votes']:
        id = vote['id']
        ipfs = vote['ipfs']
        voter = vote['voter']
        created = vote['created']
        space = vote['space']['id']
        proposal = vote['proposal']['id']
        choice = vote['choice']
        #metadata = vote['metadata']: {},
        vp = vote['vp']
        vp_by_strategy = tuple(vote['vp_by_strategy'])
        vp_state = vote['vp_state']
        row_as_series  = pd.Series(
            data=(
                id,
                ipfs,
                voter,
                created,
                space,
                proposal,
                choice,
                #metadata,
                vp,
                vp_by_strategy,
                vp_state,
                ),
            index=(
                'id',
                'ipfs',
                'voter',
                'created',
                'space',
                'proposal',
                'choice',
                #'metadata',
                'vp',
                'vp_by_strategy',
                'vp_state',
            )
        )
        row_as_series_list.append(row_as_series)
    return_df = pd.DataFrame(row_as_series_list)
    return return_df
    

In [79]:
votes_df = vote_json_to_df(vote_json)

In [80]:
votes_df

,id,ipfs,voter,created,space,proposal,choice,vp,vp_by_strategy,vp_state
0,0x142418f28d21b3ba356caa91025d45ee84558dde1e7a...,QmYpBuRKXYdLfgaCMKnzuwqEBmj9EijWTpNL5hnmzr6oxG,0x53C40473DcdFd927C4201cCFE24E314a7D7C3584,1635623597,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,88.858457,"(88.85845661274872, 0)",final
1,0x7b696f7df1ec720a1b976d06ffed895e8b7ed0cb8872...,QmUrs4tt8a49ZQPasxXvrQ2xGGWb9ZhSUTBU7rqMEE8KZj,0x0100e4f2a841e773564c333f889ab6D6Bd5FcB22,1635602262,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,8912.113703,"(753.0289283021797, 8159.084775177758)",final
2,0x1ead0f3d9ebd31a303db747fd12b89b7c701ec4d2d56...,QmdhDd2vLrG2kCrFutiRRbyNDz1zsz5xEvrt7WGv7qCg5s,0x6E07663b21bD087a53739BC2109aC8360ebB0270,1635584931,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,2,77.069316,"(77.06931592224655, 0)",final
3,0xd100c55917933d152f013829aa9c74dd7fc58ad8dfec...,QmZ2m2HS9rJ4ifkVdPuTaDHvtdaU33Qn5v4G3bZw3wXa7j,0x7f5Fe0Cf45aD50b4E80Fe66a87f692B7c006bc3e,1635574698,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,1.009999,"(1.009998885338774, 0)",final
4,0x2de599e9ea9c9217d7094fc48f6044dba3c7bc285582...,QmRUHT2ATZt7iEVgRpV99HzApKanCMXZsMHuaaMt8cqBv8,0x83C9440dc34DA00c47A0d4dC2b598d7BDB1b53F7,1635553233,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,5.694520,"(5.69451958746213, 0)",final
5,0xace7b8e7c6432d2a14425c1350329708a1551163463f...,QmeC4qVMseA4avgGfTeLBr2zh557ahGG7cKQj8TTKms89E,0x4AdA1B9D9fe28aBd9585f58cfEeD2169A39e1c6b,1635549257,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,148.161771,"(148.16177066678787, 0)",final
6,0x937907e6e9dc2dbb3956a5af44c8c526f98045f9c6aa...,QmcdZpUjhB38x1T6FeBfgZLjLKwTZQrGTedXzuffbtZPcy,0x070341aA5Ed571f0FB2c4a5641409B1A46b4961b,1635529404,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,8793.329084,"(0, 8793.329084453937)",final
7,0x0e8cce791593cd24705fa42bad14e1c934d17a14da9b...,QmfJaGHnmLdv2CTdTTk8YWy1uTnZTLrYCGxNpoS4KRRVGv,0x4f49D938c3Ad2437c52eb314F9bD7Bdb7FA58Da9,1635521733,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,104.830000,"(104.83, 0)",final
8,QmbyBUw4mfLyaamtpAzA8DitX5HxRuo6B2ikfWQFDoowL9,QmbyBUw4mfLyaamtpAzA8DitX5HxRuo6B2ikfWQFDoowL9,0x981CaC16246D9641bE07BbA6E19E304b74B173BC,1635490934,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,633.936943,"(40, 593.9369434653063)",final
9,0x8b1842fc9d30f1b7dcf3a5a5f2836af4d393e1e0d6c4...,Qmc4n4HsWExGEe6jLLKVHTu4TgiCsWuwPYdKQLdAKYLM4a,0x6a1a3eCd04DF0fF2d7cb25584052CDD173fC609B,1635461353,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,37.270088,"(37.270087876477305, 0)",final


In [48]:
vote = vote_json['data']['votes'][0]